In [19]:
# Install required libraries
# You can install these packages using pip if not installed:
# !pip install pandas sqlite3 pandasql

import sqlite3
import pandas as pd

# Set working directory
import os
os.chdir("/Users/jade/Desktop/Humana/Training")

# Load datasets using pandas
features_data = pd.read_csv("Afeatures.csv")
control_data = pd.read_csv("Controlpoint.csv")
cost_data = pd.read_csv("CostUt.csv")
demograph_data = pd.read_csv("Demographics.csv")
condition_data = pd.read_csv("mcondition.csv")
detail_data = pd.read_csv("mdetail.csv")
claims_data = pd.read_csv("mclaims.csv")
target_data = pd.read_csv("Tmembers.csv")
members_data = pd.read_csv("mdata.csv")
pharmacy_data = pd.read_csv("PU.csv")
quality_data = pd.read_csv("QD.csv")
sales_data = pd.read_csv("SC.csv")
social_data = pd.read_csv("Socialh.csv")
web_data = pd.read_csv("WA.csv")

/var/folders/wm/7c949vv90fl9xprhlvgygqw40000gn/T/ipykernel_5041/3976071838.py:19: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_data = pd.read_csv("mclaims.csv")
/var/folders/wm/7c949vv90fl9xprhlvgygqw40000gn/T/ipykernel_5041/3976071838.py:23: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  quality_data = pd.read_csv("QD.csv")


In [4]:
import re
def convert_tenure(value):
    # Check if value is a string before processing
    if isinstance(value, str):
        # If the format is 'X - Y YEARS', calculate the midpoint
        if '-' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 2:  # Ensure there are two numbers for the range
                return (float(numbers[0]) + float(numbers[1])) / 2
        # If the format is 'X+ YEARS', take the number before the '+'
        elif '+' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:  # Ensure there's at least one number
                return float(numbers[0])
        # If there's just 'X YEARS', return the number
        elif 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:
                return float(numbers[0])
        # Default case for strings that don't match the expected format
        return None
    # If value is already a float or an int, return it as is
    elif isinstance(value, (float, int)):
        return value
    # Default return for unexpected types (e.g., NaN, None)
    return None

# Applying the function to the column
members_data['tenure_band'] = members_data['tenure_band'].apply(convert_tenure)
members_data['disabled_ind'] = members_data['disabled_ind'].replace({'Y': 1, 'N': 0})
members_data['dual_eligible_ind'] = members_data['dual_eligible_ind'].replace({'Y': 1, 'N': 0})
members_data['lis_ind']= members_data['lis_ind'].replace({'Y': 1, 'N': 0})

In [5]:
from sklearn.feature_selection import VarianceThreshold
import numpy as np

# Function to remove high correlation and low variance columns
def clean_data(df, variance_threshold=0.01, correlation_threshold=0.9):
    # Step 1: Remove variables with low variance
    selector = VarianceThreshold(threshold=variance_threshold)
    df_reduced = selector.fit_transform(df)
    
    # Convert back to DataFrame to handle column names
    columns_kept = df.columns[selector.get_support()]
    df_filtered = pd.DataFrame(df_reduced, columns=columns_kept)

    # Step 2: Remove highly correlated variables
    corr_matrix = df_filtered.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
    
    # Drop highly correlated columns
    df_cleaned = df_filtered.drop(columns=to_drop)
    
    return df_cleaned

# Assuming your data is already loaded in these variables
# Apply the cleaning process to each dataset
members_data_cleaned = clean_data(members_data)
pharmacy_data_cleaned = clean_data(pharmacy_data)
social_data_cleaned = clean_data(social_data)
web_data_cleaned = clean_data(web_data)

In [6]:
print(len(members_data_cleaned))

1527904


In [7]:
print(len(members_data))

1527904


In [8]:
print(len(pharmacy_data_cleaned))
print(len(social_data_cleaned))
print(len(web_data_cleaned))

1527904
1527904
1527904


In [9]:
merged_data_cleaned = pd.merge(members_data_cleaned, pharmacy_data_cleaned, on='id', how='inner')
merged_data_cleaned = pd.merge(merged_data, social_data_cleaned, on='id', how='inner')
merged_data_cleaned = pd.merge(merged_data, web_data_cleaned, on='id', how='inner')

In [10]:
merged_data_cleaned

,consec_tenure_month,dual_eligible_ind,disabled_ind,lis_ind,id,rx_overall_pmpm_cost,rx_overall_coins_pmpm_cost,rx_overall_copay_pmpm_cost,rx_overall_deduct_pmpm_cost,rx_overall_mbr_resp_pmpm_cost,...,login_count_2,login_count_3,login_count_4,login_count_5,login_count_6,login_count_7,login_count_8,login_count_9,login_count_10,login_count_11
0,6.0,0.0,0.0,0.0,1551235.0,25.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,59.0,0.0,1.0,0.0,1643841.0,8.743333,1.855833,2.994167,1.790833,7.474167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,119.0,0.0,0.0,0.0,540296.0,510.660830,0.000000,42.635834,0.000000,76.110000,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,2.0,3.0
3,24.0,0.0,1.0,0.0,729600.0,319.575000,0.000000,83.404170,0.000000,83.404170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,60.0,0.0,0.0,0.0,716836.0,71.818340,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527899,4.0,0.0,0.0,0.0,1551651.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1527900,23.0,0.0,0.0,0.0,1584607.0,0.595833,0.000000,0.595833,0.000000,0.595833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1527901,3.0,0.0,0.0,0.0,1246390.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1527902,24.0,0.0,1.0,0.0,695942.0,96.273330,0.000000,7.374167,40.000000,33.040833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
print(len(members_data.columns))

7


In [14]:
print(len(members_data_cleaned.columns))

5


In [15]:
print(len(pharmacy_data_cleaned.columns))
print(len(social_data_cleaned.columns))
print(len(web_data_cleaned.columns))
print(len(pharmacy_data.columns))
print(len(social_data.columns))
print(len(web_data.columns))

14
35
15
17
78
15


In [17]:
merged_data = pd.merge(members_data, pharmacy_data, on='id', how='inner')
merged_data = pd.merge(merged_data, social_data, on='id', how='inner')
merged_data = pd.merge(merged_data, web_data, on='id', how='inner')

In [18]:
print(merged_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 114 entries, consec_tenure_month to login_count_11
dtypes: float64(108), int64(6)
memory usage: 1.3 GB
None


In [20]:
print(target_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Data columns (total 5 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   calendar_year             1527904 non-null  int64 
 1   product_type              1527904 non-null  object
 2   plan_category             1527904 non-null  object
 3   preventive_visit_gap_ind  1527904 non-null  int64 
 4   id                        1527904 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 58.3+ MB
None


In [31]:
new_target=target_data.drop(['product_type','calendar_year','plan_category'],axis=1)
merged_data = pd.merge(merged_data, new_target, on='id', how='inner')

In [48]:
datasets = {
    'features_data': features_data,
    'control_data': control_data,
    'cost_data': cost_data,
    'demograph_data': demograph_data,
    'condition_data': condition_data,
    'detail_data': detail_data,
    'claims_data': claims_data,
    'target_data': target_data,
    'members_data': members_data,
    'pharmacy_data': pharmacy_data,
    'quality_data': quality_data,
    'sales_data': sales_data,
    'social_data': social_data,
    'web_data': web_data
}

# Check number of rows for each dataset
for name, data in datasets.items():
    print(f"{name} has {len(data)} rows")
    print(f"{name} has {data.info()} rows")

features_data has 1527904 rows
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Data columns (total 12 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   cms_frailty_ind            1527393 non-null  float64
 1   cms_tot_ma_payment_amt     1508168 non-null  float64
 2   cms_tot_partd_payment_amt  1508168 non-null  float64
 3   atlas_recfacpth14          1526922 non-null  float64
 4   atlas_ffrpth14             1526922 non-null  float64
 5   atlas_fsrpth14             1526922 non-null  float64
 6   atlas_grocpth14            1526922 non-null  float64
 7   atlas_povertyallagespct    1526922 non-null  float64
 8   cci_score                  1527393 non-null  float64
 9   fci_score                  1527393 non-null  float64
 10  dcsi_score                 1527393 non-null  float64
 11  id                         1527904 non-null  int64  
dtypes: float64(11), int64(1)
memory usage: 

In [45]:
#condition_data
#cliams_data
#quality_data
features_data 
control_data 
cost_data 
#demograph_data 
#detail_data 
#claims_data 
target_data 
members_data 
pharmacy_data 
#quality_data 
#sales_data 
social_data 
web_data 

In [53]:
print(features_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Data columns (total 12 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   cms_frailty_ind            1527393 non-null  float64
 1   cms_tot_ma_payment_amt     1508168 non-null  float64
 2   cms_tot_partd_payment_amt  1508168 non-null  float64
 3   atlas_recfacpth14          1526922 non-null  float64
 4   atlas_ffrpth14             1526922 non-null  float64
 5   atlas_fsrpth14             1526922 non-null  float64
 6   atlas_grocpth14            1526922 non-null  float64
 7   atlas_povertyallagespct    1526922 non-null  float64
 8   cci_score                  1527393 non-null  float64
 9   fci_score                  1527393 non-null  float64
 10  dcsi_score                 1527393 non-null  float64
 11  id                         1527904 non-null  int64  
dtypes: float64(11), int64(1)
memory usage: 139.9 MB
None


In [54]:
print(control_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Data columns (total 66 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   cnt_cp_emails_0              1527393 non-null  float64
 1   cnt_cp_emails_1              1527393 non-null  float64
 2   cnt_cp_emails_2              1527393 non-null  float64
 3   cnt_cp_emails_3              1527393 non-null  float64
 4   cnt_cp_emails_4              1527393 non-null  float64
 5   cnt_cp_emails_5              1527393 non-null  float64
 6   cnt_cp_emails_6              1527393 non-null  float64
 7   cnt_cp_emails_7              1527393 non-null  float64
 8   cnt_cp_emails_8              1527393 non-null  float64
 9   cnt_cp_emails_9              1527393 non-null  float64
 10  cnt_cp_emails_10             1527393 non-null  float64
 11  cnt_cp_emails_11             1527393 non-null  float64
 12  cnt_cp_print_0               1527393 non-n

In [55]:
print(cost_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Data columns (total 36 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   nonpar_clm_ct_pmpm                  1527393 non-null  float64
 1   nonpar_allowed_pmpm_cost            1527393 non-null  float64
 2   nonpar_net_paid_pmpm_cost           1527393 non-null  float64
 3   nonpar_cob_paid_pmpm_cost           1527393 non-null  float64
 4   nonpar_coins_pmpm_cost              1527393 non-null  float64
 5   nonpar_copay_pmpm_cost              1527393 non-null  float64
 6   nonpar_deduct_pmpm_cost             1527393 non-null  float64
 7   nonpar_mbr_resp_pmpm_cost           1527393 non-null  float64
 8   nonpar_ds_clm                       1527393 non-null  float64
 9   oontwk_clm_ct_pmpm                  1527393 non-null  float64
 10  oontwk_allowed_pmpm_cost            1527393 non-null  float64
 11  oontwk_net_

In [56]:
condition_data

,cond_key,chronicity,cond_desc,hcc_model_type,cms_model_vers_cd,membership_year,id
0,226,Chronic,"Heart Failure, Except End-Stage and Acute",MEDICAL,V28,2023,993642
1,94,Chronic,Systemic Lupus Erythematosus and Other Specifi...,MEDICAL,V28,2023,993642
2,329,Chronic,"Chronic Kidney Disease, Moderate Stage 3, Exce...",MEDICAL,V28,2023,993642
3,81,Chronic,Ulcerative Colitis,MEDICAL,V28,2023,803124
4,47,Chronic,Disorders of Immunity,MEDICAL,V24,2023,803124
...,...,...,...,...,...,...,...
4009337,37,Chronic,Diabetes with Chronic Complications,MEDICAL,V28,2023,654526
4009338,238,Chronic,Specified Heart Arrhythmias,MEDICAL,V28,2023,654526
4009339,329,Chronic,"Chronic Kidney Disease, Moderate Stage 3, Exce...",MEDICAL,V28,2023,654526
4009340,38,Chronic,"Diabetes with Glycemic, Unspecified, or No Com...",MEDICAL,V28,2023,1124053


In [57]:
merged_data = pd.merge(merged_data, features_data, on='id', how='inner')
merged_data = pd.merge(merged_data, control_data, on='id', how='inner')
merged_data = pd.merge(merged_data, cost_data, on='id', how='inner')

In [58]:
print(merged_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 226 entries, consec_tenure_month to total_ip_snf_admit_days_pmpm
dtypes: float64(219), int64(7)
memory usage: 2.6 GB
None


In [60]:
demograph_data

,lang_spoken_cd,rucc_category,riskarr_downside,riskarr_upside,riskarr_rewards,riskarr_global,id
0,ENG,1-Metro,0.0,1.0,0.0,0.0,1352611
1,ENG,2-Metro,0.0,0.0,0.0,0.0,299881
2,NaN,2-Metro,0.0,0.0,0.0,0.0,1191826
3,ENG,7-Nonmetro,0.0,1.0,0.0,0.0,1772108
4,NaN,7-Nonmetro,0.0,0.0,0.0,0.0,171691
...,...,...,...,...,...,...,...
1527899,ENG,6-Nonmetro,0.0,1.0,0.0,0.0,376551
1527900,ENG,8-Nonmetro,0.0,0.0,0.0,0.0,1637412
1527901,NaN,2-Metro,0.0,0.0,0.0,0.0,338186
1527902,ENG,3-Metro,0.0,0.0,0.0,0.0,351507


In [62]:
print(demograph_data['lang_spoken_cd'].value_counts())

lang_spoken_cd
ENG    1058350
OTH      47238
SPA      15468
KOR        769
CHI        528
VIE        442
FRE        248
JPN        159
POL        108
RUS         87
CPF         76
TGL         66
YUE         64
POR         44
ITA         43
ARA         39
CMN         35
GER         10
FAS         10
PER          7
LAO          5
NAV          5
HIN          3
CRE          3
TAG          2
HMG          2
DUT          2
ZZZ          1
THA          1
BEN          1
PHI          1
MAN          1
GUJ          1
SRP          1
IRA          1
URD          1
Name: count, dtype: int64


In [63]:
print(demograph_data['rucc_category'].value_counts())

rucc_category
1-Metro       463353
2-Metro       364590
3-Metro       230472
6-Nonmetro    161116
4-Nonmetro    124668
7-Nonmetro     81417
5-Nonmetro     42660
8-Nonmetro     30477
9-Nonmetro     28612
Name: count, dtype: int64


In [59]:
condition_data

,cond_key,chronicity,cond_desc,hcc_model_type,cms_model_vers_cd,membership_year,id
0,226,Chronic,"Heart Failure, Except End-Stage and Acute",MEDICAL,V28,2023,993642
1,94,Chronic,Systemic Lupus Erythematosus and Other Specifi...,MEDICAL,V28,2023,993642
2,329,Chronic,"Chronic Kidney Disease, Moderate Stage 3, Exce...",MEDICAL,V28,2023,993642
3,81,Chronic,Ulcerative Colitis,MEDICAL,V28,2023,803124
4,47,Chronic,Disorders of Immunity,MEDICAL,V24,2023,803124
...,...,...,...,...,...,...,...
4009337,37,Chronic,Diabetes with Chronic Complications,MEDICAL,V28,2023,654526
4009338,238,Chronic,Specified Heart Arrhythmias,MEDICAL,V28,2023,654526
4009339,329,Chronic,"Chronic Kidney Disease, Moderate Stage 3, Exce...",MEDICAL,V28,2023,654526
4009340,38,Chronic,"Diabetes with Glycemic, Unspecified, or No Com...",MEDICAL,V28,2023,1124053
